In [10]:
import pandas as pd

In [11]:
df = pd.read_excel('bentonite.xlsx')

In [12]:
df.head()

,Mt.c [%],LL[%],PL [%],PI [%],Wi [%],γd [g.m-3],Pr [MPa]
0,48.0,473.9,26.6,447.3,5.95,1.457224,0.477416
1,48.0,473.9,26.6,447.3,5.95,1.467422,0.571764
2,48.0,473.9,26.6,447.3,5.95,1.504816,0.515702
3,48.0,473.9,26.6,447.3,5.95,1.494618,0.451398
4,48.0,473.9,26.6,447.3,5.95,1.589802,0.665309


In [13]:
df.columns

Index(['Mt.c [%]', 'LL[%]', 'PL [%]', 'PI [%]', 'Wi [%]', 'γd [g.m-3]',
       'Pr [MPa]'],
      dtype='object')

In [14]:
df.isnull().sum()

Mt.c [%]      0
LL[%]         0
PL [%]        0
PI [%]        0
Wi [%]        0
γd [g.m-3]    0
Pr [MPa]      0
dtype: int64

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import joblib


# Define input features and output variable
X = df.drop('Pr [MPa]', axis=1)
y = df['Pr [MPa]']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

# Create a dictionary to store the models and their scalers
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'SVR': SVR(),
    'CatBoostRegressor' : CatBoostRegressor(),
     'XGBRegressor' : XGBRegressor()
    
    
}

# Function to evaluate and save model performance
def evaluate_model(model, X_train_scaled, X_test_scaled, y_train, y_test, model_name, scaler_name):
    # Train the model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    
    # Save the model and the scaler
    joblib.dump(model, f'{model_name}_{scaler_name}.pkl')
    joblib.dump(scalers[scaler_name], f'{scaler_name}.pkl')
    
    return {
        'Model': model_name,
        'Scaler': scaler_name,
        'MSE': mse,
        'MAE': mae,
        'R2': r2,
        'CV Mean Score': np.mean(cv_scores),
        'CV Std Dev': np.std(cv_scores)
    }

# List to store evaluation results
evaluation_results = []

# Train and evaluate models with different scalers
for scaler_name, scaler in scalers.items():
    # Scale the data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    for model_name, model in models.items():
        results = evaluate_model(model, X_train_scaled, X_test_scaled, y_train, y_test, model_name, scaler_name)
        evaluation_results.append(results)
print(evaluation_results)
# Convert evaluation results to a DataFrame for better visualization
results_df = pd.DataFrame(evaluation_results)

# Save the results to a CSV file
results_df.to_csv('model_evaluation_results.csv', index=False)

results_df


Learning rate set to 0.032591
0:	learn: 3.2294646	total: 13.7ms	remaining: 13.7s
1:	learn: 3.1731096	total: 15.6ms	remaining: 7.8s
2:	learn: 3.1243372	total: 16.9ms	remaining: 5.63s
3:	learn: 3.0711787	total: 18.8ms	remaining: 4.68s
4:	learn: 3.0195329	total: 20.6ms	remaining: 4.09s
5:	learn: 2.9739596	total: 22ms	remaining: 3.65s
6:	learn: 2.9310554	total: 23.9ms	remaining: 3.39s
7:	learn: 2.8828813	total: 25.3ms	remaining: 3.14s
8:	learn: 2.8392781	total: 26.6ms	remaining: 2.93s
9:	learn: 2.8059239	total: 27.2ms	remaining: 2.7s
10:	learn: 2.7586088	total: 28.5ms	remaining: 2.56s
11:	learn: 2.7127667	total: 29.7ms	remaining: 2.45s
12:	learn: 2.6715061	total: 31.1ms	remaining: 2.36s
13:	learn: 2.6338477	total: 32.6ms	remaining: 2.29s
14:	learn: 2.5985470	total: 34ms	remaining: 2.23s
15:	learn: 2.5580915	total: 35.3ms	remaining: 2.17s
16:	learn: 2.5272978	total: 36.6ms	remaining: 2.12s
17:	learn: 2.4935314	total: 37.9ms	remaining: 2.07s
18:	learn: 2.4610974	total: 39.3ms	remaining: 2.03

241:	learn: 1.0016844	total: 698ms	remaining: 2.19s
242:	learn: 1.0009796	total: 700ms	remaining: 2.18s
243:	learn: 0.9996635	total: 703ms	remaining: 2.18s
244:	learn: 0.9983111	total: 704ms	remaining: 2.17s
245:	learn: 0.9973040	total: 706ms	remaining: 2.16s
246:	learn: 0.9952302	total: 707ms	remaining: 2.15s
247:	learn: 0.9942752	total: 708ms	remaining: 2.15s
248:	learn: 0.9922609	total: 710ms	remaining: 2.14s
249:	learn: 0.9904060	total: 711ms	remaining: 2.13s
250:	learn: 0.9885829	total: 712ms	remaining: 2.13s
251:	learn: 0.9871947	total: 714ms	remaining: 2.12s
252:	learn: 0.9857522	total: 715ms	remaining: 2.11s
253:	learn: 0.9841962	total: 717ms	remaining: 2.1s
254:	learn: 0.9824240	total: 718ms	remaining: 2.1s
255:	learn: 0.9815371	total: 720ms	remaining: 2.09s
256:	learn: 0.9808082	total: 721ms	remaining: 2.08s
257:	learn: 0.9792170	total: 722ms	remaining: 2.08s
258:	learn: 0.9785015	total: 724ms	remaining: 2.07s
259:	learn: 0.9770849	total: 725ms	remaining: 2.06s
260:	learn: 0.

529:	learn: 0.7546719	total: 1.06s	remaining: 938ms
530:	learn: 0.7534997	total: 1.06s	remaining: 936ms
531:	learn: 0.7522332	total: 1.06s	remaining: 933ms
532:	learn: 0.7519530	total: 1.06s	remaining: 930ms
533:	learn: 0.7517486	total: 1.06s	remaining: 927ms
534:	learn: 0.7509440	total: 1.06s	remaining: 925ms
535:	learn: 0.7501533	total: 1.06s	remaining: 922ms
536:	learn: 0.7499884	total: 1.06s	remaining: 919ms
537:	learn: 0.7496902	total: 1.07s	remaining: 916ms
538:	learn: 0.7488535	total: 1.07s	remaining: 913ms
539:	learn: 0.7485637	total: 1.07s	remaining: 911ms
540:	learn: 0.7476728	total: 1.07s	remaining: 908ms
541:	learn: 0.7474039	total: 1.07s	remaining: 906ms
542:	learn: 0.7466082	total: 1.07s	remaining: 903ms
543:	learn: 0.7458865	total: 1.07s	remaining: 900ms
544:	learn: 0.7454078	total: 1.07s	remaining: 898ms
545:	learn: 0.7450941	total: 1.08s	remaining: 895ms
546:	learn: 0.7447591	total: 1.08s	remaining: 892ms
547:	learn: 0.7439835	total: 1.08s	remaining: 890ms
548:	learn: 

849:	learn: 0.5731954	total: 1.43s	remaining: 252ms
850:	learn: 0.5727652	total: 1.43s	remaining: 250ms
851:	learn: 0.5723905	total: 1.43s	remaining: 248ms
852:	learn: 0.5720614	total: 1.43s	remaining: 247ms
853:	learn: 0.5717239	total: 1.43s	remaining: 245ms
854:	learn: 0.5710019	total: 1.43s	remaining: 243ms
855:	learn: 0.5702527	total: 1.43s	remaining: 241ms
856:	learn: 0.5693410	total: 1.44s	remaining: 240ms
857:	learn: 0.5686308	total: 1.44s	remaining: 238ms
858:	learn: 0.5681791	total: 1.44s	remaining: 236ms
859:	learn: 0.5675181	total: 1.44s	remaining: 234ms
860:	learn: 0.5669533	total: 1.44s	remaining: 233ms
861:	learn: 0.5665878	total: 1.44s	remaining: 231ms
862:	learn: 0.5660701	total: 1.46s	remaining: 232ms
863:	learn: 0.5653162	total: 1.46s	remaining: 230ms
864:	learn: 0.5647820	total: 1.46s	remaining: 229ms
865:	learn: 0.5646213	total: 1.47s	remaining: 227ms
866:	learn: 0.5637719	total: 1.47s	remaining: 225ms
867:	learn: 0.5633331	total: 1.47s	remaining: 223ms
868:	learn: 

87:	learn: 1.3912361	total: 95.4ms	remaining: 989ms
88:	learn: 1.3855872	total: 97.1ms	remaining: 994ms
89:	learn: 1.3782398	total: 98.4ms	remaining: 995ms
90:	learn: 1.3735333	total: 99.5ms	remaining: 994ms
91:	learn: 1.3674955	total: 101ms	remaining: 994ms
92:	learn: 1.3602488	total: 102ms	remaining: 993ms
93:	learn: 1.3543565	total: 103ms	remaining: 992ms
94:	learn: 1.3484712	total: 104ms	remaining: 990ms
95:	learn: 1.3423954	total: 105ms	remaining: 985ms
96:	learn: 1.3342999	total: 106ms	remaining: 983ms
97:	learn: 1.3280510	total: 107ms	remaining: 982ms
98:	learn: 1.3229194	total: 108ms	remaining: 981ms
99:	learn: 1.3173732	total: 109ms	remaining: 981ms
100:	learn: 1.3138265	total: 111ms	remaining: 986ms
101:	learn: 1.3080516	total: 112ms	remaining: 986ms
102:	learn: 1.3051394	total: 114ms	remaining: 990ms
103:	learn: 1.2977128	total: 115ms	remaining: 990ms
104:	learn: 1.2915734	total: 116ms	remaining: 990ms
105:	learn: 1.2855752	total: 117ms	remaining: 988ms
106:	learn: 1.2802822

...

230:	learn: 1.0104204	total: 223ms	remaining: 744ms
231:	learn: 1.0083280	total: 225ms	remaining: 744ms
232:	learn: 1.0064865	total: 226ms	remaining: 743ms
233:	learn: 1.0049096	total: 227ms	remaining: 743ms
234:	learn: 1.0040504	total: 228ms	remaining: 742ms
235:	learn: 1.0020970	total: 229ms	remaining: 741ms
236:	learn: 1.0005075	total: 230ms	remaining: 740ms
237:	learn: 0.9994155	total: 231ms	remaining: 739ms
238:	learn: 0.9970193	total: 232ms	remaining: 738ms
239:	learn: 0.9958234	total: 233ms	remaining: 737ms
240:	learn: 0.9951432	total: 234ms	remaining: 736ms
241:	learn: 0.9941879	total: 235ms	remaining: 735ms
242:	learn: 0.9930393	total: 236ms	remaining: 734ms
243:	learn: 0.9911234	total: 237ms	remaining: 733ms
244:	learn: 0.9895490	total: 238ms	remaining: 732ms
245:	learn: 0.9884737	total: 239ms	remaining: 731ms
246:	learn: 0.9859410	total: 240ms	remaining: 730ms
247:	learn: 0.9851260	total: 240ms	remaining: 729ms
248:	learn: 0.9846121	total: 241ms	remaining: 728ms
249:	learn: 

525:	learn: 0.7222736	total: 582ms	remaining: 524ms
526:	learn: 0.7212095	total: 583ms	remaining: 524ms
527:	learn: 0.7202700	total: 584ms	remaining: 522ms
528:	learn: 0.7191227	total: 586ms	remaining: 521ms
529:	learn: 0.7188546	total: 587ms	remaining: 520ms
530:	learn: 0.7180009	total: 588ms	remaining: 519ms
531:	learn: 0.7177601	total: 589ms	remaining: 518ms
532:	learn: 0.7168882	total: 590ms	remaining: 517ms
533:	learn: 0.7166797	total: 591ms	remaining: 515ms
534:	learn: 0.7153682	total: 592ms	remaining: 514ms
535:	learn: 0.7144776	total: 593ms	remaining: 513ms
536:	learn: 0.7136985	total: 593ms	remaining: 512ms
537:	learn: 0.7128378	total: 595ms	remaining: 511ms
538:	learn: 0.7126828	total: 595ms	remaining: 509ms
539:	learn: 0.7118916	total: 597ms	remaining: 508ms
540:	learn: 0.7111087	total: 597ms	remaining: 507ms
541:	learn: 0.7107715	total: 598ms	remaining: 506ms
542:	learn: 0.7099981	total: 599ms	remaining: 505ms
543:	learn: 0.7097032	total: 601ms	remaining: 503ms
544:	learn: 

713:	learn: 0.6034086	total: 765ms	remaining: 307ms
714:	learn: 0.6025601	total: 767ms	remaining: 306ms
715:	learn: 0.6019718	total: 768ms	remaining: 305ms
716:	learn: 0.6013902	total: 769ms	remaining: 303ms
717:	learn: 0.6007868	total: 770ms	remaining: 302ms
718:	learn: 0.6002153	total: 771ms	remaining: 301ms
719:	learn: 0.5996522	total: 772ms	remaining: 300ms
720:	learn: 0.5995393	total: 773ms	remaining: 299ms
721:	learn: 0.5994039	total: 774ms	remaining: 298ms
722:	learn: 0.5988490	total: 775ms	remaining: 297ms
723:	learn: 0.5986096	total: 776ms	remaining: 296ms
724:	learn: 0.5985215	total: 777ms	remaining: 295ms
725:	learn: 0.5976976	total: 778ms	remaining: 294ms
726:	learn: 0.5974226	total: 779ms	remaining: 292ms
727:	learn: 0.5968564	total: 780ms	remaining: 291ms
728:	learn: 0.5966884	total: 781ms	remaining: 290ms
729:	learn: 0.5961367	total: 782ms	remaining: 289ms
730:	learn: 0.5956126	total: 783ms	remaining: 288ms
731:	learn: 0.5946463	total: 784ms	remaining: 287ms
732:	learn: 

896:	learn: 0.5130685	total: 945ms	remaining: 108ms
897:	learn: 0.5124091	total: 946ms	remaining: 107ms
898:	learn: 0.5123315	total: 947ms	remaining: 106ms
899:	learn: 0.5118118	total: 948ms	remaining: 105ms
900:	learn: 0.5116970	total: 950ms	remaining: 104ms
901:	learn: 0.5111449	total: 951ms	remaining: 103ms
902:	learn: 0.5103320	total: 952ms	remaining: 102ms
903:	learn: 0.5097623	total: 953ms	remaining: 101ms
904:	learn: 0.5096211	total: 954ms	remaining: 100ms
905:	learn: 0.5091111	total: 955ms	remaining: 99ms
906:	learn: 0.5089009	total: 956ms	remaining: 98ms
907:	learn: 0.5082478	total: 957ms	remaining: 96.9ms
908:	learn: 0.5079480	total: 958ms	remaining: 95.9ms
909:	learn: 0.5078508	total: 959ms	remaining: 94.8ms
910:	learn: 0.5074053	total: 959ms	remaining: 93.7ms
911:	learn: 0.5071404	total: 960ms	remaining: 92.7ms
912:	learn: 0.5067361	total: 961ms	remaining: 91.6ms
913:	learn: 0.5060474	total: 962ms	remaining: 90.6ms
914:	learn: 0.5057008	total: 963ms	remaining: 89.5ms
915:	l

,Model,Scaler,MSE,MAE,R2,CV Mean Score,CV Std Dev
0,LinearRegression,StandardScaler,4.080467,1.470164,0.647805,0.537590,0.062768
1,Ridge,StandardScaler,4.082049,1.470347,0.647668,0.538398,0.060724
2,Lasso,StandardScaler,9.135073,2.403676,0.211529,0.169417,0.074035
3,DecisionTreeRegressor,StandardScaler,3.780183,1.073559,0.673723,0.665988,0.030238
4,RandomForestRegressor,StandardScaler,2.207994,0.898644,0.809423,0.744878,0.035697
5,GradientBoostingRegressor,StandardScaler,1.776227,0.878788,0.846689,0.757639,0.014330
6,SVR,StandardScaler,2.959641,1.028104,0.744546,0.644634,0.059630
7,CatBoostRegressor,StandardScaler,2.043885,0.849383,0.823587,0.770492,0.025991
8,XGBRegressor,StandardScaler,2.953435,0.932668,0.745082,0.742244,0.033735
9,LinearRegression,MinMaxScaler,4.080467,1.470164,0.647805,0.538195,0.063508
